In [1]:
from tqdm.auto import tqdm
import json
from typing import List
import random
import re
import ast

/gaueko0/users/jetxaniz007/alpaca-lora-mt/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def encode_string(text):
    return text.replace("\r", r"\r").replace("\n", r"\n").replace("\t", r"\t")


def decode_string(text):
    return text.replace("\\r", "\r").replace("\\n", "\n").replace("\\t", "\t")


def prepare_data_for_translation(original_json: str, output_path: str):
    with open(original_json, "r", encoding="utf-8") as original_file, open(
        output_path, "w", encoding="utf-8"
    ) as output_file:
        data = json.load(original_file)
        for example in tqdm(data):
            instruction = example["instruction"].strip()
            model_input = example["input"].strip()
            model_output = example["output"].strip()
            if len(instruction) > 0:
                print(
                    encode_string(instruction),
                    file=output_file,
                )
            if len(model_input) > 0:
                print(
                    encode_string(model_input),
                    file=output_file,
                )
            if len(model_output) > 0:
                print(
                    encode_string(model_output),
                    file=output_file,
                )


def is_numbers(text):
    return re.search("[a-zA-Z]", text) is None


def is_code(text):
    try:
        ast.parse(text)
    except SyntaxError:
        return False
    return True


def build_from_translation(original_json: str, translations: str, output_json: str):
    with open(original_json, "r", encoding="utf-8") as original_file, open(
        translations, "r", encoding="utf-8"
    ) as translation_file, open(output_json, "w", encoding="utf-8") as output_file:
        data = json.load(original_file)
        translations = translation_file.readlines()
        translations = [decode_string(line) for line in translations]
        output_dict = []
        i = 0
        for example in tqdm(data):
            translated_example = {"instruction": "", "input": "", "output": ""}
            instruction = example["instruction"].strip()
            model_input = example["input"].strip()
            model_output = example["output"].strip()

            if len(instruction) > 0:
                if is_numbers(instruction) or is_code(instruction):
                    translated_example["instruction"] = instruction
                else:
                    translated_example["instruction"] = translations[i]
                i += 1
            if len(model_input) > 0:
                if is_numbers(model_input) or is_code(model_input):
                    translated_example["input"] = model_input
                else:
                    translated_example["input"] = translations[i]
                i += 1
            if len(model_output) > 0:
                if is_numbers(model_output) or is_code(model_output):
                    translated_example["output"] = model_output
                else:
                    translated_example["output"] = translations[i]
                i += 1

            output_dict.append(translated_example)

        json.dump(output_dict, output_file, indent=4, ensure_ascii=False)


def merge_examples(json_paths: List[str], output_path: str):
    merged_examples = []
    for json_path in json_paths:
        with open(json_path, "r", encoding="utf-8") as json_path:
            data = json.load(json_path)
            merged_examples.extend(data)

    # shuffle
    random.shuffle(merged_examples)
    with open(output_path, "w", encoding="utf-8") as output_file:
        json.dump(merged_examples, output_file, indent=4, ensure_ascii=False)

In [3]:
prepare_data_for_translation(
    "../data/alpaca_data_cleaned.json", "../data/en.sentences.txt"
)

  0%|          | 0/51942 [00:00<?, ?it/s]

In [3]:
path = "/ikerlariak/igarcia945/alpaca-lora-mt/data/"

In [12]:
build_from_translation(
    path + "../data/alpaca_data_cleaned.json",
    path + "../data/en.sentences.txt",
    "../data/en.sanitycheck.json",
)

100%|██████████| 51942/51942 [00:01<00:00, 32704.05it/s]


In [5]:
lang_codes = {
    "spa_Latn": "es",
    "glg_Latn": "gl",
    "eus_Latn": "eu",
    "cat_Latn": "ca",
    "por_Latn": "pt",
    "ast_Latn": "at",
}

for code3, code2 in lang_codes.items():
    build_from_translation(
        path + "../data/alpaca_data_cleaned.json",
        path + f"../data/{code3}.sentences.txt",
        f"../data/alpaca_data_cleaned_{code2}.json",
    )

100%|██████████| 51942/51942 [00:01<00:00, 34706.76it/s]


In [13]:
build_from_translation(
    path + "../data/alpaca_data_cleaned.json",
    path + "../data/spa_Latn.sentences.txt",
    "../data/alpaca_data_cleaned_es.json",
)

100%|██████████| 51942/51942 [00:01<00:00, 30501.18it/s]


In [14]:
build_from_translation(
    path + "../data/alpaca_data_cleaned.json",
    path + "../data/eus_Latn.sentences.txt",
    "../data/alpaca_data_cleaned_eu.json",
)

100%|██████████| 51942/51942 [00:01<00:00, 32112.65it/s]


False